## Initiate surface passive tracer flux 

In [1]:
# Load modules
%pylab inline
from netCDF4 import Dataset # to work with NetCDF files
import numpy as np
import matplotlib.pyplot as plt # to generate plots
from mpl_toolkits.basemap import Basemap # plot on map projections
from os.path import expanduser
home = expanduser("~") # Get users home directory
import os # operating system interface
import scipy as sp
import xarray as xr
from xarray.ufuncs import square, sqrt
import scipy.interpolate
import scipy.ndimage




Populating the interactive namespace from numpy and matplotlib


In [2]:
#extract the MOM model grid info - 50 and 75 vertical levels
hgrid_file ='/g/data1/v45/pas561/mom/archive/gfdl_nyf_1080/mom025_ocean_grid.nc'
vgrid_file ='/g/data1/v45/pas561/mom/archive/gfdl_nyf_1080/ocean.ncra501-520.nc'

# Extract the variables
nc = Dataset(hgrid_file, mode='r') # file handle, open in read only mode
geolon_t = nc.variables['geolon_t'][:]
geolat_t = nc.variables['geolat_t'][:]
area_t = nc.variables['area_t'][:]
dxt = nc.variables['dxt'][:]
dyt = nc.variables['dyt'][:]
kmu = nc.variables['kmu'][:]
kmt = nc.variables['kmt'][:]
ht = nc.variables['ht'][:]
fcor = nc.variables['f_coriolis'][:]
nc.close() # close the file
print geolon_t.shape

fh=fcor/ht
print fh.shape

nc = Dataset(vgrid_file, mode='r') # file handle, open in read only mode
sw_ocean = nc.variables['sw_ocean'][:]
st_ocean = nc.variables['st_ocean'][:]
yt_ocean = nc.variables['yt_ocean'][:]
yu_ocean = nc.variables['yu_ocean'][:]
xt_ocean = nc.variables['xt_ocean'][:]
potrho = nc.variables['potrho'][:]
nc.close() # close the file
print st_ocean.shape
print potrho.shape
print yt_ocean.shape



(1080, 1440)
(1080, 1440)
(50,)
(80,)
(1080,)


In [3]:
#A = xr.DataArray(np.arange(100))
#A.cumsum(axis=0)
print (xr.__version__)

#my xarray version is old, no cumsum. Should be version 0.9.2 or newer?

0.10.0


In [4]:
CntrlDir = '/g/data1/v45/pas561/mom/archive/gfdl_nyf_1080_PTrace/'
#PertDir = '/g/data1/v45/pas561/mom/archive/wombat_jun16z_PI/'

#ExpDirs=[CntrlDir, PertDir]
print CntrlDir

/g/data1/v45/pas561/mom/archive/gfdl_nyf_1080_PTrace/


In [5]:
Ocean_File = os.path.join(CntrlDir,'output*/ocean__*.nc')
print Ocean_File
ff=xr.open_mfdataset(Ocean_File,engine='netcdf4',concat_dim='time',decode_times=False)
print ff.passive_patch1.shape
print ff.dzt.shape




/g/data1/v45/pas561/mom/archive/gfdl_nyf_1080_PTrace/output*/ocean__*.nc
(6, 50, 1080, 1440)
(6, 50, 1080, 1440)


In [ ]:
#print ff.passive_patch1[0,:,500,500].load()

In [8]:
net_pass=ff.passive_patch1[0,:,:,:].sum('yt_ocean').sum('xt_ocean').sum('st_ocean').load()
print net_pass
net_pass=ff.passive_patch1[0,:,:,:].load()
print net_pass.shape
#print net_pass
tt=np.ma.sum(net_pass)
print tt
#print 'estimate: ', net_pass*1.30642202560882944E+18

<xarray.DataArray 'passive_patch1' ()>
array(955654.7630851169)
Coordinates:
    time     float64 1.825e+05
(50, 1080, 1440)
<xarray.DataArray 'passive_patch1' ()>
array(955654.763072692)
Coordinates:
    time     float64 1.825e+05


In [ ]:
#number of time-steps in a 15 days run
#dt=1800
#ts=15*24*60*60/1800
#print ts

#number of surface ocean grid points
#cnt =0
#print ht.shape
#for i in range(1440):
#        for j in range(1080):
#            if ht[j,i]>0:
#                cnt+=1
#print cnt

#divide by Cp and thickness of upper cell (annual mean approx of z*) to get W/m2
#J/kg-1/K-1
##cnt = (ht > 0).sum()
#print cnt

In [15]:
#calculate global dV

#dvt=dzt[0,:,:,:]*dxt*dyt
print ff.dzt.shape
dvt=ff.dzt[:,:,:,:].load()*dxt*dyt
print dvt.shape

print np.ma.sum(dvt[0,:,:,:])
#print np.ma.sum(dvt[3,:,:,:])
#print np.ma.sum(dvt[5,:,:,:])
#1.30642202560882944E+18 m^3 from MOM.out
#correct

#check single value
#print dvt[0,:,500,100]
#print dzt[0,:,500,100]
#print dxt[500,100]
#print dyt[500,100]
#print dxt[500,100]*dyt[500,100]
#print dxt[500,100]*dyt[500,100]*ff.dzt[0,:,500,100].load()

(6, 50, 1080, 1440)
(6, 50, 1080, 1440)
<xarray.DataArray 'dzt' ()>
array(1.306403328433147e+18)
Coordinates:
    time     float64 1.825e+05


In [16]:
#calcute tracer*dv at each grid cell
#save as p2

print ff.passive_patch1.shape
p2=ff.passive_patch1[:,:,:,:].load()*dvt
print p2.shape

p3=np.zeros(p2.shape[0])
for t in range(p2.shape[0]):
    print t
    p3[t]=np.ma.sum(p2[t,:,:,:])*1035
    print p3[t]

print 'global int: ', p3

#print 'global int/Vol: ', tt/np.ma.sum(dvt[:,:,:])


(6, 50, 1080, 1440)
(6, 50, 1080, 1440)
0
3.77843502789e+18
1
3.77843489583e+18
2
3.77843522829e+18
3
3.77843522957e+18
4
3.77843524919e+18
5
3.77843534188e+18
global int:  [  3.77843503e+18   3.77843490e+18   3.77843523e+18   3.77843523e+18
   3.77843525e+18   3.77843534e+18]


In [18]:
print np.diff(p3)

[ -1.32053296e+11   3.32457789e+11   1.27912192e+09   1.96188483e+10
   9.26879053e+10]


In [14]:
#calcute tracer*dv at each grid cell
#save as p2
dvt=ff.dzt[0,:,:,:].load()*dxt*dyt

print ff.passive_patch1.shape
p2=ff.passive_patch1[0,:,:,:].load()
print p2.shape
print 'tracer sum:', np.ma.sum(p2)
print dvt.shape
print 'volume:', np.ma.sum(dvt)
print 'sum(pp)*sum(dvt): ', np.ma.sum(p2)*np.ma.sum(dvt)

#p2=ff.passive_patch1[:,:,:,:].load()*ff.dzt[:,:,:,:].load()
#p2=ff.passive_patch1[0,:,:,:].load()*dvt
#print p2.shape
#print p2[:,500,100]

p3=p2*dvt
print p3.shape

tt=np.ma.sum(p3)*1035
print 'global int: ', tt

#print 'global int/Vol: ', tt/np.ma.sum(dvt[:,:,:])


(6, 50, 1080, 1440)
(50, 1080, 1440)
tracer sum: <xarray.DataArray 'passive_patch1' ()>
array(955654.763072692)
Coordinates:
    time     float64 1.825e+05
(50, 1080, 1440)
volume: <xarray.DataArray 'dzt' ()>
array(1.306403328433147e+18)
Coordinates:
    time     float64 1.825e+05
sum(pp)*sum(dvt):  <xarray.DataArray ()>
array(1.248470563311155e+24)
Coordinates:
    time     float64 1.825e+05
(50, 1080, 1440)
global int:  <xarray.DataArray ()>
array(3.7784350278857436e+18)
Coordinates:
    time     float64 1.825e+05


In [ ]:
#print p2.shape
#print dxt.shape
#print dyt.shape
#p2=p2*dxt*dyt
#print p2.shape

In [8]:
#check values of p2=dv*tracer
c1=ff.passive_patch1[0,:,500,100].load()
c2=ff.dzt[0,:,500,100].load()
print 'tracer val: ', c1
print 'dzt: ', c2
print 'area: ',dxt[500,100]*dyt[500,100]
print 'check ', c1*c2*dxt[500,100]*dyt[500,100]

#check should equal p2 values
print 'p2 ',p2[0,:,500,100]

tracer val:  <xarray.DataArray 'passive_patch1' (st_ocean: 50)>
array([ 0.244353,  0.230175,  0.222547,  0.217883,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan])
Coordinates:
    xt_ocean  float64 -254.9
    yt_ocean  float64 0.625
  * st_ocean  (st_ocean) float64 5.034 15.1 25.22 35.36 45.58 55.85 66.26 ...
    geolon_t  float64 -254.9
    geolat_t  float64 0.625
    time      float64 1.825e+05
Attributes:
    long_name:      passive (patch_1)
    units:          dimensionless
 

In [9]:
#globally integrate the tracer*dv

#print p2[0,:,500,100]
print p2.shape
p3=np.zeros(p2.shape[0])
for t in range(p2.shape[0]):
    print t
    p3[t]=np.ma.sum(p2[t,:,:,:])
    print p3[t]

print p3



(6, 50, 1080, 1440)
0
3.65066186269e+15
1
3.6506617351e+15
2
3.65066205632e+15
3
3.65066205755e+15
4
3.65066207651e+15
5
3.65066216606e+15
[  3.65066186e+15   3.65066174e+15   3.65066206e+15   3.65066206e+15
   3.65066208e+15   3.65066217e+15]


In [10]:
#p3 shounld increase as a global constant
print np.diff(p3)

-127587725.0
321215255.5
1235866.5
18955409.0
89553531.5


In [13]:
#try another way
p1=ff.passive_patch1[0,:,:,:].load()
dz1=ff.dzt[0,:,:,:].load()

print p1.shape, dz1.shape


(50, 1080, 1440) (50, 1080, 1440)


In [17]:
p1_int = np.zeros(6)
print p1_int.shape

for t in range(6):
    p1=ff.passive_patch1[t,:,:,:].load()
    dz1=ff.dzt[t,:,:,:].load()

    tmp=p1*dz1*dxt*dyt
    print tmp.shape
    p1_int[t] = np.ma.sum(tmp)
    print p1_int[t]

#globally integrate
#p1_int=0
#for k in range(50):
##    print k
#    for j in range(1080):
#        for i in range(1440):
#            p1_int=p1_int+p1[k,j,i]*dz1[k,j,i]*dxt[j,i]*dyt[j,i]

#print p1_int

(6,)
(50, 1080, 1440)
3.65066186269e+15
(50, 1080, 1440)
3.6506617351e+15
(50, 1080, 1440)
3.65066205632e+15
(50, 1080, 1440)
3.65066205755e+15
(50, 1080, 1440)
3.65066207651e+15
(50, 1080, 1440)
3.65066216606e+15


In [18]:
print np.diff(p1_int)

[ -1.27587725e+08   3.21215256e+08   1.23586600e+06   1.89554090e+07
   8.95535315e+07]


In [19]:
#try another way
p1_int = np.zeros(6)
print p1_int.shape

for t in range(6):
    for k in range(50):
        print k
        p1=ff.passive_patch1[t,k,:,:].load()
        dz1=ff.dzt[t,k,:,:].load()

        tmp=p1*dz1*dxt*dyt
        print tmp.shape
        p1_int[t] = p1_int[t]+np.ma.sum(tmp)
        

p1_int[t]

(6,)
0
(1080, 1440)
1
(1080, 1440)
2
(1080, 1440)
3
(1080, 1440)
4
(1080, 1440)
5
(1080, 1440)
6
(1080, 1440)
7
(1080, 1440)
8
(1080, 1440)
9
(1080, 1440)
10
(1080, 1440)
11
(1080, 1440)
12
(1080, 1440)
13
(1080, 1440)
14
(1080, 1440)
15
(1080, 1440)
16
(1080, 1440)
17
(1080, 1440)
18
(1080, 1440)
19
(1080, 1440)
20
(1080, 1440)
21
(1080, 1440)
22
(1080, 1440)
23
(1080, 1440)
24
(1080, 1440)
25
(1080, 1440)
26
(1080, 1440)
27
(1080, 1440)
28
(1080, 1440)
29
(1080, 1440)
30
(1080, 1440)
31
(1080, 1440)
32
(1080, 1440)
33
(1080, 1440)
34
(1080, 1440)
35
(1080, 1440)
36
(1080, 1440)
37
(1080, 1440)
38
(1080, 1440)
39
(1080, 1440)
40
(1080, 1440)
41
(1080, 1440)
42
(1080, 1440)
43
(1080, 1440)
44
(1080, 1440)
45
(1080, 1440)
46
(1080, 1440)
47
(1080, 1440)
48
(1080, 1440)
49
(1080, 1440)
0
(1080, 1440)
1
(1080, 1440)
2
(1080, 1440)
3
(1080, 1440)
4
(1080, 1440)
5
(1080, 1440)
6
(1080, 1440)
7
(1080, 1440)
8
(1080, 1440)
9
(1080, 1440)
10
(1080, 1440)
11
(1080, 1440)
12
(1080, 1440)
13
(108

3650662166133544.0

In [20]:
print p1_int

[  3.65066186e+15   3.65066174e+15   3.65066206e+15   3.65066206e+15
   3.65066208e+15   3.65066217e+15]


In [21]:
print np.diff(p1_int)

[ -1.27574326e+08   3.21214398e+08   1.23246600e+06   1.89570115e+07
   8.95551635e+07]
